In [0]:
import tensorflow as tf

In [0]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [0]:
def tensor2Img(tensor):
  tensor = tensor * 255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor) > 3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [0]:
contentPath = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg',
                                       'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
stylePath = tf.keras.utils.get_file('kandinsky5.jpg',
                                    'https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

In [0]:
def loadImg(path):
  maxSide = 512
  img = tf.io.read_file(path)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  longSide = max(shape)
  scale = maxSide / longSide

  newShape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, newShape)
  img = img[tf.newaxis, :]
  return img

In [0]:
def imshow(img, title=None):
  if len(img.shape) > 3:
    img = tf.squeeze(img, axis=0)
  plt.imshow(img)
  if title:
    plt.title(title)

In [0]:
contentImg = loadImg(contentPath)
styleImg = loadImg(stylePath)

plt.subplot(1, 2, 1)
imshow(contentImg, 'Content')

plt.subplot(1, 2, 2)
imshow(styleImg, 'Style')

In [0]:
import tensorflow_hub as hub
hubMod = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')
stylizedImg = hubMod(tf.constant(contentImg), tf.constant(styleImg))[0]
tensor2Img(stylizedImg)

In [0]:
x = tf.keras.applications.vgg19.preprocess_input(contentImg * 255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
predictProbs = vgg(x)
predictProbs.shape

In [0]:
predictTop5 = tf.keras.applications.vgg19.decode_predictions(predictProbs.numpy())[0]
[(className, prob) for (number, className, prob) in predictTop5]

In [0]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

print()
for layer in vgg.layers:
  print(layer.name)

In [0]:
contentLayers = ['block5_conv2']
styleLayers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1'
]
nCL = len(contentLayers)
nSL = len(styleLayers)

In [0]:
def vggLayers(layerNames):
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable=False

  outputs = [vgg.get_layer(name).output for name in layerNames]

  model = tf.keras.Model([vgg.input], outputs)
  return model

In [0]:
styleExtract = vggLayers(styleLayers)
styleOutputs = styleExtract(styleImg * 255)

for name, output in zip(styleLayers, styleOutputs):
  print(name)
  print('    shape: ', output.numpy().shape)
  print('    min: ', output.numpy().min())
  print('    max: ', output.numpy().max())
  print('    mean: ', output.numpy().mean())
  print()

In [0]:
def gramMat(input):
  res = tf.linalg.einsum('bijc,bijd->bcd', input, input)
  inputShape = tf.shape(input)
  nLocs = tf.cast(inputShape[1]*inputShape[2], tf.float32)
  return res / nLocs

In [0]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, styleLayers, contentLayers):
    super(StyleContentModel, self).__init__()
    self.vgg = vggLayers(styleLayers + contentLayers)
    self.styleLayers = styleLayers
    self.contentLayers = contentLayers
    self.nSL = len(styleLayers)
    self.vgg.trainable = False

  def call(self, inputs):
    inputs = inputs * 255.0
    preprocessedInput = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessedInput)
    styleOutputs, contentOutputs = (outputs[:self.nSL],
                                    outputs[self.nSL:])
    styleOutputs = [gramMat(styleOut) for styleOut in styleOutputs]
    contentDict = {contentName:value for contentName, value
                   in zip(self.contentLayers, contentOutputs)
    }
    styleDict = {styleName:value for styleName, value
                 in zip(self.styleLayers, styleOutputs)
    }
    return {'content':contentDict, 'style':styleDict}

In [0]:
extractor = StyleContentModel(styleLayers, contentLayers)

res = extractor(tf.constant(contentImg))
print('Styles')
for name, output in sorted(res['style'].items()):
  print(name)
  print('    shape: ', output.numpy().shape)
  print('    min: ', output.numpy().min())
  print('    max: ', output.numpy().max())
  print('    mean: ', output.numpy().mean())
  print()

print('Contents:')
for name, output in sorted(res['content'].items()):
  print(name)
  print('    shape: ', output.numpy().shape)
  print('    min: ', output.numpy().min())
  print('    max: ', output.numpy().max())
  print('    mean: ', output.numpy().mean())
  print()

In [0]:
styleTargs = extractor(styleImg)['style']
contentTargs = extractor(contentImg)['content']

In [0]:
img = tf.Variable(contentImg)

In [0]:
def clip01(img):
  return tf.clip_by_value(img, clip_value_min=0.0, clip_value_max=1.0)

In [0]:
opt = tf.optimizers.Adam(learning_rate = .02, beta_1=.99, epsilon=1e-1)

In [0]:
wS = 1e-2
wC = 1e4

In [0]:
def styleContentLoss(outputs):
  styleOut = outputs['style']
  contentOut = outputs['content']
  styleLoss = tf.add_n([tf.reduce_mean((styleOut[name] - styleTargs[name]) ** 2)
                        for name in styleOut.keys()])
  styleLoss *= wS / nSL
  contentLoss = tf.add_n([tf.reduce_mean((contentOut[name] - contentTargs[name]) ** 2)
                          for name in contentOut.keys()])
  contentLoss = wC / nCL
  loss = styleLoss + contentLoss
  return loss

In [0]:
@tf.function()
def trainStep(img):
  with tf.GradientTape() as tape:
    outputs = extractor(img)
    loss = styleContentLoss(outputs)
  
  grad = tape.gradient(loss, img)
  opt.apply_gradients([(grad, img)])
  img.assign(clip01(img))

In [0]:
import time
start = time.time()

epochs = 10
stepsPerEpoch = 100

step = 0
for n in range(epochs):
  for m in range(stepsPerEpoch):
    step += 1
    trainStep(img)
    print('.', end='')
  display.clear_output(wait=True)
  display.display(tensor2Img(img))
  print('Train step: {}'.format(step))

end = time.time()
print('Time elapsed: {:.1f}'.format(end-start))